In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.metrics.pairwise import linear_kernel

In [4]:
movies = pd.read_csv("movies_metadata.csv",
                    usecols=["id","overview","title","vote_average","vote_count","release_date"])


In [5]:
movies.head()

,id,overview,release_date,title,vote_average,vote_count
0,862,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,7.7,5415.0
1,8844,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,6.9,2413.0
2,15602,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,6.5,92.0
3,31357,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,6.1,34.0
4,11862,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,5.7,173.0
